In [1]:
try:
    import google.colab
    inColab = True
except ImportError:
    inColab = False

### Install Library

In [2]:
# Hugging Face에서 개발한 도구로, 딥러닝 모델의 훈련과 추론 속도를 높이기 위한 최적화
# PEFT( Parameter-Efficient Fine-Tuning) 딥러닝 모델의 파라미터를 효율적으로 미세 조정하는 방법 제공
# bitsandbytes 딥러닝 모델을 더 작은 메모리 풋프린트로 실행할 수 있게 해주는 라이브러리
# transformers 사전 학습된 모델들을 제공하는 도구, 대형 모델을 쉽게 사용할 수 있게 해줌
# trl(Transformers Reinforcement Learning) 사전 학습된 트랜스포머 모델에 강화 학습 적용하여 특정 작업에 맞게 모델을 더 정밀하게 조정하는 데 사용
# dataset 다양한 데이터셋을 쉽게 로드, 처리, 변환, 분석할 수 있는 라이브러리
# -U 옵션: 지정된 패키지가 이미 설치되어 있는 경우에도 최신 버전으로 업그레이드
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 122.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
import torch  # 딥러닝 라이브러리 중 하나
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer  # 지도학습하겠다
from datetime import datetime
import huggingface_hub

In [4]:
if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [5]:
%cd /content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/LLM/datasets

/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/중간고사/LLM/datasets


In [6]:
huggingface_hub.login()

# 1. set model and dataset

### ★★★ 수정 포인트 ★★★

In [77]:
# set base model path
base_model = "beomi/Llama-3-Open-Ko-8B"
# base_model = "upstage/SOLAR-10.7B-Instruct-v1.0"
# base_model = "meta-llama/Meta-Llama-3-8B-Instruct"
# base_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
# base_model = model_id = "google/gemma-7b-it"
dataset_name = "limjh12/law"
datasetCommon = load_dataset(dataset_name, split="train")

In [79]:
datasetCommon[40]

{'instruction': '학교보건법을 개정하여 새롭게 관리하려고 하는 건 뭐야',
 'output': '인조잔디 운동장의 유해물질',
 'input': ''}

In [80]:
def convert_to_alpaca_format(dataset):
    instruction = dataset['instruction']
    output = dataset['output']

    # 'text' 컬럼 생성
    text = f"### Instruction:\n{instruction}\n\n###Input:\n \n\n###Output:\n{output}"
    dataset["text"] = text
    return dataset

# 'text' 컬럼 생성

In [81]:
datasetCommon = datasetCommon.map(convert_to_alpaca_format, remove_columns=['instruction', 'input', 'output'])

In [82]:
datasetCommon

Dataset({
    features: ['text'],
    num_rows: 5456
})

In [83]:
# 데이터 건수 50개로 제한 (직접 데이터를 확인하고 가장 노동법과 유사한 데이터 선정함 - 40~90의 값들이 노동법 관련 데이터)
dataset = datasetCommon.select(range(40, 90))
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 50
})


# 2. Config efficient fine-tuning with low-rank adaptation.

In [84]:
#
torch.cuda.get_device_capability()[0]

7

In [85]:
# 현재 사용 중인 GPU의 주요 아키텍처 버전을 반환 8버전 이상 시 bfloat16 활용
# NVIDIA Ampere 아키텍처 이상 시에만 처리
# 정확도를 위하여 float 16이 타도록 강제설정 8->10 ★ 향후 변경 필요
if torch.cuda.get_device_capability()[0] >= 10:
    # 고속 attention 메커니즘을 구현하는 라이브러리
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16


# BitsAndBytesConfig 객체활용 양자화 설정
quant_config = BitsAndBytesConfig(
    # 모델을 4비트 양자화하여 로드할지 여부 결정 - 4비트 양자화 : 소수점을 버리고 4비트만 쓸게
    load_in_4bit=True,
    # 양자화 방법 (nf4: Non-Uniform Quantization, "nf4","fp16 등))
    bnb_4bit_quant_type="nf4",
    # (4비트 양자화 시 사용할 데이터 타입, "torch.float16, bfloat16, float32 등)
    bnb_4bit_compute_dtype=torch_dtype,
    # 이중 양자화 사용여부 (이중 양자화는 양자화 과정에서 정밀도 높이기 위해 활용, 대신 더 연산은 복잡)
    bnb_4bit_use_double_quant=False,
)

# 3. Load Pre-trained Language Model

In [86]:
model = AutoModelForCausalLM.from_pretrained(
    # 불러올 모델 정의
    base_model,
    # 모델 양자화 설정값
    quantization_config=quant_config,
    # 모델의 레이어를 할당할 장치 ("":0 -> 전체 모델을 GPU 0에 할당, "auto"는 알아서, "{"layer_0":0, ... 형태로 레이어별 할당 가능)
    # device_map={"": 0}  # T4 GPU로는 터짐
    device_map="auto"  # GPU랑 CPU를 같이 써서 불러오는 느낌
)
# 캐시 사용 여부 (모델 출력 매번 새로 계산)
# 새로운 데이터에 대한 계산 시 저장하는게 불필요함 또한 True 시 캐시 저장할 메모리 사용하여 메모리 사용량 증가함
model.config.use_cache = False
# 모델읜 pretraining tensor parallelism 설정 1인경우 병렬처리 안하고 단일장치 활용 2이상 시 병렬 처리를 분산 가능
# 병렬 처리 시 추가적인 메모리 오버헤드 발생 가능함
model.config.pretraining_tp = 1

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

# 4. Load Pre-trained Language Model Tokenizer

In [16]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
# Must add EOS_TOKEN at response last line
tokenizer.pad_token = tokenizer.eos_token

# ★수정 포인트!!! 기존 # tokenizer.padding_side = "right"
EOS_TOKEN = tokenizer.eos_token
def prompt_eos(sample):
    sample['text'] = sample['text']+EOS_TOKEN
    return sample
datasetCommon = dataset.map(prompt_eos)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [17]:
datasetCommon[1]

{'text': '### Instruction:\n국민대타협기구 합의문을 기본으로 한 논의에서 어떤 결론을 도출했어\n\n###Input:\n \n\n###Output:\n공무원연금개혁안<|end_of_text|>'}

In [18]:
import gc
# # Flush memory
# del trainer, model
gc.collect()
torch.cuda.empty_cache()

# 5. Config training parameter for LoRA (Parameter-Efficient Fine-Tuning (PEFT)

https://huggingface.co/docs/peft/conceptual_guides/lora

In [19]:
# training 환경 설정
peft_params = LoraConfig(
    # lora 방식의 튜닝에서 저차원 병렬 레이어 학습 ( 값이 크면 저차원 병렬 레이어 영향이 커지고 작으면 줄어듬, 기본 16, 1~128)
    lora_alpha=16,
    # 과적합 방지 (0.1에서 증가 시킴, 0.0~0.3)
    lora_dropout=0.1,
    # LoRA 저차원 공간의 차원 수 (값이 커지면 파라미터 공간 커지지만 계산비용 증가 4, 8, 16, 32, 64 로 튜닝, 4~128)
    r=64,
    # LoRA 모델의 바이어스 파라미터 적용여부 None = 미적용 ("none" or "all", "lora_only" 설정 가능)
    bias="none",
    # LoLA 모델의 작업 유형 (GPT와 같은 인과모델) ("CASUAL_LM": GPT, "SEQ_CLASSSIFICATION": 시퀀스 분류, "TOKEN_CLASSIFICATION": 토큰 분류, "SEQ_2_SEQ_LM": 번역모델)
    task_type="CAUSAL_LM",
)

In [20]:
training_params = TrainingArguments(
    # 학습 결과와 체크포인트를 저장할 디렉토리 경로
    output_dir="./results",
    # 전체 데이터셋을 반복학습 주기 (1~50 정도 활용)
    num_train_epochs=15,
    # 각 GPU 장치에서 사용하는 배치 크기 (배치 크기에 따라 가중치 업데이트 1~64)
    # 배치크기는 모델히 한번에 처리하는 데이터 샘플의 수
    per_device_train_batch_size=3,
    #그래디언트 누적을 통한 배치 크기 증가 (여러 작은 배치를 합쳐서 효과적으로 큰 배치 크기를 적용 가능) *4 적용 시 그래디언트 누적 4개 후 한번에 가중치 업데이트
    # 배치크기가 4배 증가한 효과를 가지지만 실제 메모리 사용은 배치 1개 크기만 사용
    gradient_accumulation_steps=1,
    # 사용할 옵티마이저 설정 ("adamw, paged_adamw_32bit","adamw_torch")
    optim="paged_adamw_32bit",
    # 학습 중 모델 체크포인트 저장 주기 (스템 수)
    save_steps=25,
    # 학습 중 로그를 기록할 주기(스텝 수)
    logging_steps=25,
    # 학습률 설정 (모델의 파라미터 업데이트 시 스템 크기, 1e-6 ~ 1e-2))
    learning_rate=2e-4,
    # 모델의 가중치에 패널티 적용하여 과적합 방지 (0.0은 정규화 안함, 0.0 ~ 0.1)
    weight_decay=0.001,
    # 16비트 부동 소수점(FP16) 연산을 사용하여 메모리 사용량과 계산속도 개선 (사용시 정밀도 감소, 32비트 부동소수점보다 절반)
    fp16=False,
    # bfloat16 연산을 사용하여 학습 FP16대비 더 넓은 지수 범위 (사용시 정밀도 감소, 32비트 부동소수점보다 절반)
    bf16=False,
    # (그래디언트 클리핑을 위한 최대 노름 값 설정, 0.1 ~ 10)
    # 그래디언트 클리핑은 그래디언트의 크기가 너무 커서 학습이 불안정해지는 것을 방지하기 위한 기법 0.3 초과하지 않도록 함 (크면 가중치가 너무 크게업데이트됨)
    max_grad_norm=0.3,
    # 전체 학습단계 수 (-1 인 경우 epochs, 이외에는 스탭수,  -1~100000)
    max_steps=-1,
    # 학습률 warmup비율 설정 (학습 초기에 학습률을 서서히 증가시켜 안정적인 학습, 0.0~0.5)
    # 너무 높으면 학습률이 너무 늦어짐
    warmup_ratio=0.03,
    # 배치 내 시쿼스 길이를 그룹화하여 패딩을 최소화 (데이터 시권스길이 유사한것 끼리 그룹화하여 메모리 효율설 높임)
    group_by_length=True,
    # 학습률 스케줄러의 유형 설정 ("linear","cosine","constant","polynomial" 등) * constant는 학습률을 일정하게 유지
    lr_scheduler_type="constant",
    # 학습 로그를 보고할 플랫폼 설정 (아웃풋 디렉토리 참고) -> tensorboard --logdir=./results/runs * wandb로 설정 가능
    report_to="tensorboard"
)


# 6. Train Model

In [21]:
trainer = SFTTrainer(
    # 학습할 모델
    model=model,
    # 모델 학습에 사용할 데이터셋
    train_dataset=datasetCommon,
    #  PEFT(파라미터 효율적 미세 조정) 설정 정의
    peft_config=peft_params,
    # 데이터셋에서 학습 데이터셋 텍스트 필드 이름
    # dataset_text_field="text",
    # 입력 시퀀스 최대 길이 (128 ~ 1024) * 길이가 길수록 더많은 컨텍스트를 모델에 제공가능 단, 메모리 사용량 증가
    # max_seq_length=None,
    # 모델과 함께 사용할 토크나이저
    # tokenizer=tokenizer,
    args=training_params,
    # 입력 시퀀스 패킹여부 (패킹 시 짧고/긴 시퀀스를 혼합하여 배치 처리 서능 개선)
    # packing=False,
)
trainer.train()

Converting train dataset to ChatML:   0%|          | 0/50 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
25,2.527400
50,1.533900
75,1.103500
100,0.766200
125,0.511000
150,0.385900
175,0.283700
200,0.168000
225,0.135700
250,0.136800


TrainOutput(global_step=255, training_loss=0.743037425069248, metrics={'train_runtime': 556.7061, 'train_samples_per_second': 1.347, 'train_steps_per_second': 0.458, 'total_flos': 1628941029507072.0, 'train_loss': 0.743037425069248})

# 7. Verify

In [22]:
# text-generation 작업을 위한 파이프라인 객체 생성
pipe = pipeline(
    # 작업 유형: 텍스트 생성
    task="text-generation",
    # 사용할 모델
    model=model,
    # 모델의 토크나이저
    tokenizer=tokenizer,
    # 모델에 전달할 추가 인자: float16 데이터 타입을 사용하여 모델을 로드 ("bfloat16"으로 학습했다면 bfloat16 사용)
    model_kwargs={"torch_dtype": torch.float16},
    # 입력 텍스트가 너무 길 경우 잘라내기
    truncation=True
)

Device set to use cuda:0


In [71]:
def extract_response_llama3(question):
    # 사용자 질문을 포함하는 메시지 리스트를 생성
    messages = [
        # 시스템 메시지: 일반적으로 시스템의 지침이나 상태값 저장  (당신은 말을 따듯하게 해주세요.)
        {
            "role": "system",
            "content": "너는 노동법 전문가야. 정확한 답을 제출해야 해. 그리고 마지막에는 '감사합니다'를 출력해"
        },
        # 사용자 메시지: 사용자 질문을 포함
        {
            "role": "user",
            "content": question
        },
    ]

    # 메시지를 토크나이저를 사용하여 모델의 입력 형식에 맞게 변환
    prompt = pipe.tokenizer.apply_chat_template(
        messages,                             # 메시지 리스트
        tokenize=False,                       # 토크나이즈를 하지 않음
        add_generation_prompt=True            # 텍스트 생성을 위한 프롬프트 추가
    )

    # 텍스트 생성 종료 토큰 ID 목록
    terminators = [
        pipe.tokenizer.eos_token_id,                        # End Of Sequence 토큰 ID
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")  # 사용자 정의 종료 토큰 ID
    ]

    outputs = pipe(
        prompt,                          # 생성할 프롬프트
        max_new_tokens=256,              # 생성할 최대 토큰 수
        eos_token_id=terminators,        # 텍스트 생성 종료를 위한 토큰 ID
        do_sample=False,                  # 샘플링을 사용하여 다음 토큰(텍스트) 생성 (True 시 토큰 무작위 선택 하여 창의성  높임, False 시 창의성 낮춤)
        temperature=0.1,                 # 샘플링의 온도 설정: 낮은 온도는 자유도를 주지 않음 (0보다 크고1보다 작아야함)  (0은 학습한 대로만 대답함. 0.9는 창조해서 대답)
        # top_p=0.9,                       # 상위 90%의 확률을 가진 토큰중에서만 무작위로 선택 do_sample true인 경우에만 설정
        num_return_sequences=1           # 생성할 시퀀스의 수: 한 개만 생성
    )

    generated_text = outputs[0]['generated_text']        # 생성된 텍스트를 추출
    response_lines = generated_text.strip().split('\n')  # 텍스트를 줄 단위로 분리
    meaningful_response = response_lines[-1]             # 마지막 줄을 응답으로 선택

    # print("=== Prompt ===")
    # print(prompt)
    # print("==============")

    return meaningful_response

In [72]:
question = "공무원연금제도의 개혁 방안을 논의하기 위해 만들어진 조직은 뭐야?"
response = extract_response_llama3(question)
print(response)

국민대타협기구


In [75]:
question = "언제 처음으로 임금피크제가 시행됐어?"
response = extract_response_llama3(question)
print(response)

2003년


In [ ]:
# 튜닝한 어댑터만 저장
savePath = "./models/llama_tune_common"
trainer.save_model(savePath)